# Tests

## Test Clustering Loss

In [1]:
import os
import importlib
import numpy as np
import tensorflow as tf

try:
    tf.config.set_visible_devices([tf.config.list_physical_devices('GPU')[0]], 'GPU')
except:
    pass

2023-04-18 10:45:22.835597: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-18 10:45:22.932492: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-18 10:45:22.956552: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/etinfo/users2/qlanglois/.local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is requir

In [2]:

# Copyright (C) 2022 yui-mhcp project's author. All rights reserved.
# Licenced under the Affero GPL v3 Licence (the "Licence").
# you may not use this file except in compliance with the License.
# See the "LICENCE" file at the root of the directory for the licence information.
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import tensorflow as tf

from utils.distance import distance

class GE2ESegLoss(tf.keras.losses.Loss):
    def __init__(self,
                 mode = 'softmax',
                 distance_metric    = 'cosine',
                 
                 init_w = 1.,
                 init_b = 0.,

                 name = 'ge2e_seg_loss',
                 
                 ** kwargs
                ):
        assert mode in ('softmax', 'contrast')
        
        super().__init__(name = name, ** kwargs)
        self.mode   = mode
        self.distance_metric    = distance_metric
        
        if mode == 'softmax':
            self.loss_fn = self.softmax_loss
        else:
            self.loss_fn = self.contrast_loss
        
        self.w = tf.Variable(init_w, trainable = True, dtype = tf.float32, name = 'weight')
        self.b = tf.Variable(init_b, trainable = True, dtype = tf.float32, name = 'bias')
    
    @property
    def variables(self):
        return [self.w, self.b]
    
    @property
    def trainable_variables(self):
        return [self.w, self.b]

    def similarity_matrix(self, mask, embeddings):
        embeddings = tf.gather_nd(embeddings, mask.indices[:, :-1])
        ids        = tf.cast(mask.indices[:, -1], tf.int32)

        centroids, centroid_ids  = compute_centroids(embeddings, ids)

        return ids, distance(
            embeddings,
            centroids,
            method  = self.distance_metric,
            force_distance  = False,
            max_matrix_size = -1,
            as_matrix       = True
        )
    
    def softmax_loss(self, idx, similarity_matrix):
        similarity_matrix = tf.nn.softmax(similarity_matrix, axis = -1)

        return tf.keras.losses.sparse_categorical_crossentropy(
            idx, similarity_matrix
        )
    
    def contrast_loss(self, idx, similarity_matrix):
        target_matrix = tf.one_hot(idx, depth = tf.shape(similarity_matrix)[-1])
        return tf.reduce_mean(tf.reshape(tf.keras.losses.binary_crossentropy(
            tf.reshape(target_matrix, [-1, 1]), tf.sigmoid(tf.reshape(similarity_matrix, [-1, 1]))
        ), [-1, tf.shape(similarity_matrix)[-1]]), axis = -1)
    
    def call(self, y_true, y_pred):
        ids, similarity_matrix = self.similarity_matrix(y_true, y_pred)

        return self.loss_fn(ids, cos_sim_matrix)

    def get_config(self):
        config = super().get_config()
        config.update({
            'mode'  : self.mode,
            'init_w'    : self.w.value().numpy(),
            'init_b'    : self.b.value().numpy(),
            'distance_metric'   : self.distance_metric
        })
        return config

In [7]:
mask = tf.sparse.SparseTensor(
    indices = [
        [1, 1, 1],
        [2, 1, 1],
        [2, 2, 1],
        [1, 2, 2],
        [2, 0, 2],
        [3, 1, 2]
    ],
    values = [1] * 6,
    dense_shape = (4, 4, 3)
)
embeddings = np.zeros((4, 4, 2))
for i, j in mask.indices[:, :2].numpy(): embeddings[i, j] = [i, j]
embeddings = tf.cast(embeddings, tf.float32)

from utils import compute_centroids

def distance_matrix(mask, embeddings):
    embeddings = tf.gather_nd(embeddings, mask.indices[:, :-1])
    ids        = tf.cast(mask.indices[:, -1], tf.int32)
    
    tf.print(tf.shape(embeddings))
    
    centroid_ids, centroids  = compute_centroids(embeddings, ids)
    
    return embeddings, ids, centroids, centroid_ids

distance_matrix(mask, embeddings)

mask = tf.sparse.reorder(mask)
print(tf.sparse.to_dense(mask))
print(tf.sparse.to_dense(tf.cast(1, mask.dtype) - mask))

[6 2]
tf.Tensor(
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 1 0]
  [0 0 1]
  [0 0 0]]

 [[0 0 1]
  [0 1 0]
  [0 1 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 1]
  [0 0 0]
  [0 0 0]]], shape=(4, 4, 3), dtype=int32)


ValueError: Attempt to convert a value (<tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fe6301545e0>) with an unsupported type (<class 'tensorflow.python.framework.sparse_tensor.SparseTensor'>) to a Tensor.

## Test Dice Loss

In [1]:
import os
import importlib
import numpy as np
import tensorflow as tf

from custom_train_objects.losses import dice_loss
from utils.med_utils import load_medical_image, load_medical_seg

try:
    tf.config.set_visible_devices([tf.config.list_physical_devices('GPU')[0]], 'GPU')
except:
    pass

path  = '/storage/Totalsegmentator_dataset/s0001'

#image = tf.expand_dims(load_medical_image(os.path.join(path, 'ct.nii.gz'))[0], axis = 0)
mask  = tf.sparse.expand_dims(load_medical_seg(os.path.join(path, 'masks.npz'))[0], axis = 0)

print('Image shape : {} - mask shape : {}'.format((), mask.shape))

2023-04-17 09:47:15.169956: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-17 09:47:15.265040: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-17 09:47:15.289095: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/etinfo/users2/qlanglois/.local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is requir

Image shape : () - mask shape : (1, 249, 188, 213, 104)


2023-04-17 09:47:25.299098: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-17 09:47:25.670536: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14783 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:17:00.0, compute capability: 7.5


In [ ]:
importlib.reload(dice_loss)

loss = dice_loss.DiceLoss(skip_empty_frames = False, skip_empty_labels = True, smoothing = 0.01)

dense_mask = tf.sparse.to_dense(tf.cast(mask, tf.float32))

empty_labels = tf.sparse.reduce_sum(tf.sparse.reshape(mask, [1, -1, 104]), axis = 1) == 0
empty_labels = tf.cast(tf.reshape(empty_labels, [1, 1, 1, 1, 104]), tf.float32)

print(empty_labels)

print(loss(mask, dense_mask).numpy())
print(loss(mask, dense_mask * 0.5).numpy())
print(loss(mask, dense_mask + 0.25 * empty_labels).numpy())

In [ ]:
%timeit loss(mask, dense_mask + 0.25 * empty_labels)
%timeit loss(dense_mask, dense_mask + 0.25 * empty_labels)
%timeit loss(tf.sparse.to_dense(mask), dense_mask + 0.25 * empty_labels)
